In [133]:
from dotenv import load_dotenv
import os
import base64
from requests import post, get, put
import json

import random

# load environment file(.env) ...that are in the same directory of this python file...
load_dotenv()

True

In [134]:
# from .env file ...we want value of 'CLIENT_ID'...
client_id = os.getenv("CLIENT_ID")

# from .env file ...we want value of 'CLIENT_SECRET'...
client_secret = os.getenv("CLIENT_SECRET")

# print(client_id,client_secret)

In [135]:
# Getting authorization token (access token)

def get_token():
    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")
    
    # End-point URL ...where we need to send the request to get the token
    url = "https://accounts.spotify.com/api/token"
    
    # This headers dictionary will be used as headers in an HTTP request..
    headers = {
        # Server use this Authorization header to authenticate the request...
        "Authorization": "Basic " + auth_base64,
        
        # Content type = 'application/x-www-form-urlencoded' indicates that the data we are sending in the HTTP request is in the
        # form of URL-encoded data.... this is basically referring to the data which has grant_type as one key...because typically
        # client credentials are url-encoded....example of urlencoded is  something like... name=John+Doe&age=25&birthdate=23-09-2002
        "Content-Type": "application/x-www-form-urlencoded"        
    }
    
    # This part is for authorization..where the client_credentials refers to the client credentials like CLIENT_ID and CLIENT_SECRET as
    # the grant_type is client_credentials...which means it tells the server that this application has been granted the access to the
    # resources through it's(client) client_credentials..so server authenticate if it is correct or not...when client make request..
    data = {"grant_type":"client_credentials"}
    
    # Sending HTTP post request to the above url endpoint
    result = post(url,headers=headers,data=data)
    
    # the response from the server is in the json format...so we convert the content of result into a dictionary..
    json_result = json.loads(result.content)
    
    # Now we search for the key - "access_token" in the dictonary.. and store it value..
    token = json_result["access_token"]
    return token;



token = get_token()
print(token)




# Authorization header ...we are using the Authorization as Bearer...so we need to provide the access token along with it...
def get_auth_header(token) :
    return {"Authorization": "Bearer " + token}


# auth_header = get_auth_header(token)
# print(auth_header)





BQBIasPNP51ieSuZqVK0yb6lY2sft3-KgRc85deh7T4BGueKklmBN3eJ078MvW6dZy_Auu-bQUdqjAB4goAfWnDwjcpk_ev9HmYkz96x8LrUDzF1IvM


## Now making different requests to Spotify-API using the access_token

## `New Releases`

### Get New Releases

In [136]:
def get_user_saved_albums(access_token):
    
    headers = get_auth_header(token)
    url  = "https://api.spotify.com/v1/browse/new-releases"
    
    response = get(url,headers = headers)
    result = json.loads(response.content)
    
    return result['albums']


new_releases = get_user_saved_albums(token)

for item in new_releases['items']:
    print(item['name'])
    
    

Come Home to Me
Lutt Putt Gaya (From "Dunki")
Arjan Vailly (From "ANIMAL")
Glass House
Guntur Kaaram
Kotabommali P.S
Kali Kali Zulfon Ke
NEW LIFE
Hua Main (From "ANIMAL")
Sunoh (From "The Archies")
Leke Prabhu Ka Naam (From "Tiger 3")
this is what space feels like
JALSA 2.0 (From "Mission Raniganj: The Great Bharat Rescue")
Saanware Aijaiyo
The Great Indian Family
Naina Milayke
Nasha (From "Sukhee")
Jawan
Jaane Jaan - Title Track (From "Jaane Jaan")
Gangs Of Godavari


## `Artist`

#### Searching for a particular `Artist` by the `Artist_name`....

In [137]:

def search_for_artist(token, artist_name) :
    
    url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    query = f"?q={artist_name}&type=artist&limit=1"
    
    query_url = url + query
    
    # This will be a json file...
    result = get(query_url,headers=headers)
    
    # json_result will be a dictionary....but we need to access key-"items" from key-"artists"...so
    json_result = json.loads(result.content)["artists"]["items"]
    
    if len(json_result) == 0 :
        print("No artist with this name exists....")
        return None
    
    return json_result[0]
    

result = search_for_artist(token, "Arctic Monkeys")
print(result["name"])
    
artist_id = result["id"]
print(artist_id)



Arctic Monkeys
7Ln80lUS6He07XvHI8qqHH


### Fetching the `top tracks` of the `Artist` through his/her `Artist_id`...

In [138]:
# Pre-requisite is the above cell code of getting the artist from its name...as it is necessary to get the id of the artist...you can
# also directly get the artist id of the artist through Spotify

# function to get songs from artist id
def get_songs_by_artist(token, artist_id) :
    url = f"https://api.spotify.com/v1/artists/{artist_id}/top-tracks?country=US"
    headers = get_auth_header(token)
    
    result = get(url, headers=headers)
    json_result = json.loads(result.content)["tracks"]
    return json_result


songs = get_songs_by_artist(token,artist_id)
# print(songs)


for idx,song in enumerate(songs) :
    print(f"{idx+1}.{song['name']}")

1.I Wanna Be Yours
2.505
3.Why'd You Only Call Me When You're High?
4.Do I Wanna Know?
5.R U Mine?
6.Fluorescent Adolescent
7.Snap Out Of It
8.Arabella
9.Knee Socks
10.Mardy Bum


## Here's how the "Bearer" authentication scheme works:

1.  `Obtaining an Access Token:` 
    - Before using the "Bearer" scheme, the client (such as a web application or API client) typically goes
through an authentication process to obtain an access token. 
    - This access token is issued by an authorization server and represents the
client's permission to access certain resources.

2.  `Including the Access Token:` 
    - Once the client has an access token, it includes it in the "Authorization" header of its HTTP requests.The header looks like this:
        > Authorization: Bearer <access_token>
    
    The word "Bearer" is followed by a space.
    <access_token> represents the actual access token obtained during the authentication process.

3. `Server Validation:` When the server receives the HTTP request with the "Authorization" header using the "Bearer" scheme, it knows that the value after "Bearer" is an access token. The server validates the token to ensure it's valid, has not expired, and has the necessary permissions to access the requested resource.

4.  `Access Grant:` 
    - If the token is valid and authorized, the server grants access to the requested resource. 
    - If the token is invalid or lacks the necessary permissions, the server denies access and responds accordingly.

The "Bearer" authentication scheme is commonly used in conjunction with OAuth 2.0 for securing APIs and web services. OAuth 2.0 grants access tokens to clients, and clients use the "Bearer" scheme to present these tokens when accessing protected resources. It's a simple and widely adopted method for transmitting access tokens in HTTP requests for authentication and authorization purposes.

## `Playlist`

### Adding `Custom` Playlist `Cover Image`

In [139]:

# Now as we are accessing the user profile...so first we need to get the permission from the user...so once the user give permission
# to share his information ....then only we can access his information...

# Like you must have seen ....when we install a new app....after opening it ask us some permission....same thing is here...if user
# grants the permission to share his data...then only we can use this code...similarly for other actions that requires user permission
# ...can only be done once the user provides permission...you can read more of this functions for which we need user persmission can 
# be seen on the Official Spotify API Documentation....

playlist_id = '7LXHR2MPB7ox0xLalBX9G2'
image_path = 'playlist.jpg'


# function to add the custom cover image to the playlist
def add_custom_cover_to_playlist(access_token, playlist_id, image_path):
    
    headers = {
        "Authorization": "Bearer " + token,
        'Content-Type': 'image/jpeg'
    }
    
    url = f"https://api.spotify.com/v1/playlists/{playlist_id}/images"
    
    
    # encoding the image to base64
    with open(image_path, 'rb') as image_file:
        image_data = image_file.read()
        base64_image = base64.b64encode(image_data).decode('utf-8')
        
    
    # payload will store our encoded image
    payload = {
        'image' : base64_image
    }
    
    result = put(url=url, headers=headers, data=payload)
    
    
    if result.status_code == 202:
        print("Custom cover image added successfully!!")
    else:
        print("⚠️Failed to add custom cover image⚠️")
    

    return result.status_code
    

output = add_custom_cover_to_playlist(token,playlist_id,image_path)
output
    

⚠️Failed to add custom cover image⚠️


401

#### Get `Playlist Cover image`

In [140]:

playlist_id = "1meWCvg9F5hk0Aw6CSmfpP"

def get_playlist_cover_image(access_token, playlist_id):
    headers = get_auth_header(token)
    
    url = f"https://api.spotify.com/v1/playlists/{playlist_id}/images"
    
    result = get(url, headers=headers)
    
    json_result = json.loads(result.content)
    
    return json_result


output = get_playlist_cover_image(token, playlist_id)
output

[{'height': 640,
  'url': 'https://mosaic.scdn.co/640/ab67616d0000b2734121faee8df82c526cbab2beab67616d0000b273a69f71a8794e2d867a52f98fab67616d0000b273e419ccba0baa8bd3f3d7abf2ab67616d0000b273e8b066f70c206551210d902b',
  'width': 640},
 {'height': 300,
  'url': 'https://mosaic.scdn.co/300/ab67616d0000b2734121faee8df82c526cbab2beab67616d0000b273a69f71a8794e2d867a52f98fab67616d0000b273e419ccba0baa8bd3f3d7abf2ab67616d0000b273e8b066f70c206551210d902b',
  'width': 300},
 {'height': 60,
  'url': 'https://mosaic.scdn.co/60/ab67616d0000b2734121faee8df82c526cbab2beab67616d0000b273a69f71a8794e2d867a52f98fab67616d0000b273e419ccba0baa8bd3f3d7abf2ab67616d0000b273e8b066f70c206551210d902b',
  'width': 60}]

### Getting the playlist from the `playlist_id`

In [141]:
# Function to get the playlist through playlist id....

def get_playlist_through_id(access_token,playlist_id):
    headers = get_auth_header(token)
    url = "https://api.spotify.com/v1/playlists/"
    
    query_url = url + playlist_id
    
    result = get(query_url,headers=headers)
    
    json_result = json.loads(result.content)
    
    return json_result
    


result = get_playlist_through_id(token,playlist_id)

playlist_name = result['name']
number_of_tracks = result['tracks']['total']

print("Playlist Name:", playlist_name)
print("Number of Tracks:", number_of_tracks)


Playlist Name: Surprise Songs
Number of Tracks: 25


##### `Bonus Section` - 
##### Here i have created different playlist of songs  on the basis of different emotions like `"joy"`, `"sadness"`, `"anger"`, `"fear"`, `"love"` & `"surprise"`

##### Now on the basis of emotions we will fetch different songs from the playlist...each time we run it we will get 7 random songs from the playlist(for a particular emotion)....

In [142]:

# Function to acessing the playlist id based on the emotion


def select_playlist_based_on_emotion(emotion):
    playlist_id = ""
    
    if(emotion == "joy"):
        playlist_id = "1CsKkNjlnIhkbz21FtMEZF"
    elif(emotion == "sadness"):
        playlist_id = "6nxPNnmSE0d5WlplUsa5L3"
    elif(emotion == "anger"):
        playlist_id = "0a4Hr64HWlxekayZ8wnWqx"
    elif (emotion == "fear"):
        playlist_id = "7LXHR2MPB7ox0xLalBX9G2"
    elif(emotion == "love"):
        playlist_id = "0hNT4Mw8aLUhD74LOPAmuh"
    elif(emotion == "surprise"):
        playlist_id = "1meWCvg9F5hk0Aw6CSmfpP"
    else:
        playlist_id = ""

    
    return playlist_id



playlist_id = select_playlist_based_on_emotion("fear")
print(playlist_id)

7LXHR2MPB7ox0xLalBX9G2


In [143]:
playlist_id

'7LXHR2MPB7ox0xLalBX9G2'

In [144]:

# getting playlist items (songs)

def get_songs_from_playlist(access_token,playlist_id):
    
    headers = get_auth_header(token)
    
    url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"
    
    result = get(url,headers=headers)
    
    json_result = json.loads(result.content)
    
    tracks = json_result['items']
    
    return tracks
    


songs = []
artists = []

    
tracks = get_songs_from_playlist(token,playlist_id)
# print(tracks)

if tracks:    
    for track in tracks:
        track_name = track["track"]["name"]
        songs.append(track_name)
        artist_name = track["track"]["artists"][0]["name"]
        artists.append(artist_name)
        
        
        #print(f"{track_name} by {artist_name}")
else:
    print("No tracks found in the playlist.")
    



# print(songs)
# print(artists)



len_songs = len(songs)
len_artist = len(artists)

numbers = 7   # number of random numbers to generate


lower_bound = 0
upper_bound = len_songs

# generating a list of random numbers within the range [0,len_songs)
random_numbers = random.sample(range(lower_bound,upper_bound),numbers)

# print(random_numbers)

for idx in random_numbers:
    print(f"{songs[idx]} by {artists[idx]}")



Church Bell Toll - Horror Sound Effect by Ultimate Horror Sounds
Ghostly Voices - Horror Sound Effect by Ultimate Horror Sounds
Fangs Spectral by Halloween Sound Effects
Loud Scream by Halloween Sound Effects
Creepy Cyclops by Horror Music Collection
Voice of Fear by Halloween Sound Effects
Haunted Waltz by Halloween Music


### Getting Songs Based On Genres

In [145]:
# Getting Songs from single Genre


def get_songs_by_genre(access_token, genre):
    
    headers = get_auth_header(token)
    url = "https://api.spotify.com/v1/recommendations"
    
    params = {
        "seed_genres":genre,
        "limit":10
    }
    
    response = get(url,headers=headers, params=params)
    
    result = json.loads(response.content)
    
    return result



result = get_songs_by_genre(token,"punk")
song_names = [track['name'] for track in result['tracks']]

for songs in song_names:
    print(songs)

Going Out
Black Sap Scriptures
Welcome to the Black Parade
I Dance to Trance in Garbage Bag Pants
Ignorance
Dear Maria, Count Me In
Cigarettes & Saints
King of the King of Beers
Mondays
Lust For Life


In [146]:

# Getting Songs from Multiple Genre

def get_songs_by_genre_list(access_token, genres):
    headers = get_auth_header(token)
    url = "https://api.spotify.com/v1/recommendations"
    limit = 5  # Number of songs per genre
    all_results = []
    
    for genre in genres:
        params = {
            "seed_genres": genre,
            "limit": limit
        }
        
        response = get(url, headers=headers, params=params)
        result = json.loads(response.content)
        
        if 'tracks' in result:
            song_names = [track['name'] for track in result['tracks']]
            all_results.extend(song_names)
        else:
            continue
        
    return all_results


# You can take input from the user in the frontend...tell him to pick his any 3 favouirte genres...then display random songs from the
# selected genre...add those genre to this below genres list...and make API call...
genres = ["acoustic", "pop", "romance"]
result = get_songs_by_genre_list(token, genres)


for song in result:
    print(song)

Miserable At Best
Down In A Hole - Live at the Majestic Theatre, Brooklyn, NY - April 1996
Why Am I Crying
Samson
Sex On Fire
pick up the phone
Paris
Tears (feat. Louisa Johnson)
Stay Together
Why You Always Hatin?
U + Ur Hand
Run to You
The Climb
All of Me - Tiësto's Birthday Treatment Remix - Radio Edit
Let's Get Married
